<a href="https://colab.research.google.com/github/pksvv/income-qualification/blob/master/income-qual3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!!git clone https://github.com/Simplilearn-Edu/Machine-Learning--Projects.git

["Cloning into 'Machine-Learning--Projects'...",
 'remote: Enumerating objects: 50, done.\x1b[K',
 'remote: Counting objects:   2% (1/50)   \x1b[K',
 'remote: Counting objects:   4% (2/50)   \x1b[K',
 'remote: Counting objects:   6% (3/50)   \x1b[K',
 'remote: Counting objects:   8% (4/50)   \x1b[K',
 'remote: Counting objects:  10% (5/50)   \x1b[K',
 'remote: Counting objects:  12% (6/50)   \x1b[K',
 'remote: Counting objects:  14% (7/50)   \x1b[K',
 'remote: Counting objects:  16% (8/50)   \x1b[K',
 'remote: Counting objects:  18% (9/50)   \x1b[K',
 'remote: Counting objects:  20% (10/50)   \x1b[K',
 'remote: Counting objects:  22% (11/50)   \x1b[K',
 'remote: Counting objects:  24% (12/50)   \x1b[K',
 'remote: Counting objects:  26% (13/50)   \x1b[K',
 'remote: Counting objects:  28% (14/50)   \x1b[K',
 'remote: Counting objects:  30% (15/50)   \x1b[K',
 'remote: Counting objects:  32% (16/50)   \x1b[K',
 'remote: Counting objects:  34% (17/50)   \x1b[K',
 'remote: Counting objects:

In [4]:
!ls Machine-Learning--Projects/Projects/'Projects for Submission'/'Project 2 - Income Qualification'

'Dataset for the project.zip'  'Income Qualification.txt'


In [5]:
!unzip Machine-Learning--Projects/Projects/'Projects for Submission'/'Project 2 - Income Qualification'/'Dataset for the project.zip'

Archive:  Machine-Learning--Projects/Projects/Projects for Submission/Project 2 - Income Qualification/Dataset for the project.zip
   creating: Dataset for the project/
  inflating: Dataset for the project/test.csv  
  inflating: Dataset for the project/train.csv  


In [17]:
#!mv 'Dataset for the project' data
!ls data

test.csv  train.csv


In [11]:
!pip install catboost

     |████████████████████████████████| 61.1MB 1.2MB/s 


In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import gc
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore',category = DeprecationWarning)
warnings.simplefilter(action='ignore',category = DeprecationWarning)


from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, RepeatedKFold, GroupKFold
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import ADASYN
import category_encoders as ce
import lightgbm as lgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from time import time

import scipy.stats as st
from sklearn.pipeline import Pipeline
from tempfile import mkdtemp
from shutil import rmtree

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


import os
print(os.listdir("data"))

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings(record=True) as w:
    # Cause all warnings to always be triggered.
    warnings.simplefilter("always")
    # Trigger a warning.
    fxn()
    # Verify some things
    assert len(w) == 1
    assert issubclass(w[-1].category, DeprecationWarning)
    assert "deprecated" in str(w[-1].message)

['train.csv', 'test.csv']


In [0]:
def dprint(*args, **kwargs):
    print("[{}] ".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")) + \
        " ".join(map(str,args)), **kwargs)

id_name = 'Id'
target_name = 'Target'

In [0]:
# Load data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [21]:
train['is_test'] = 0
test['is_test'] = 1
df_all = pd.concat([train, test], axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
dprint('Clean features...')
cols = ['dependency']
for c in tqdm(cols):
    x = df_all[c].values
    strs = []
    for i, v in enumerate(x):
        try:
            val = float(v)
        except:
            strs.append(v)
            val = np.nan
        x[i] = val
    strs = np.unique(strs)

    for s in strs:
        df_all[c + '_' + s] = df_all[c].apply(lambda x: 1 if x == s else 0)

    df_all[c] = x
    df_all[c] = df_all[c].astype(float)
dprint("Done.")

100%|██████████| 1/1 [00:00<00:00, 16.16it/s]

[2019-06-05 09:51] Clean features...
[2019-06-05 09:51] Done.


In [23]:
dprint("Extracting features...")
def extract_features(df):
    df['bedrooms_to_rooms'] = df['bedrooms']/df['rooms']
    df['rent_to_rooms'] = df['v2a1']/df['rooms']
    df['rent_to_bedrooms'] = df['v2a1']/df['bedrooms']
    df['tamhog_to_rooms'] = df['tamhog']/df['rooms'] # tamhog - size of the household
    df['tamhog_to_bedrooms'] = df['tamhog']/df['bedrooms']
    df['r4t3_to_tamhog'] = df['r4t3']/df['tamhog'] # r4t3 - Total persons in the household
    df['r4t3_to_rooms'] = df['r4t3']/df['rooms'] # r4t3 - Total persons in the household
    df['r4t3_to_bedrooms'] = df['r4t3']/df['bedrooms']
    df['rent_to_r4t3'] = df['v2a1']/df['r4t3']
    df['v2a1_to_r4t3'] = df['v2a1']/(df['r4t3'] - df['r4t1'])
    df['hhsize_to_rooms'] = df['hhsize']/df['rooms']
    df['hhsize_to_bedrooms'] = df['hhsize']/df['bedrooms']
    df['rent_to_hhsize'] = df['v2a1']/df['hhsize']
    df['qmobilephone_to_r4t3'] = df['qmobilephone']/df['r4t3']
    df['qmobilephone_to_v18q1'] = df['qmobilephone']/df['v18q1']
    

extract_features(train)
extract_features(test)
dprint("Done.")         

[2019-06-05 09:51] Extracting features...
[2019-06-05 09:51] Done.


In [0]:
from sklearn.preprocessing import LabelEncoder

def encode_data(df):
   
    yes_no_map = {'no': 0, 'yes': 1}
    
    df['dependency'] = df['dependency'].replace(yes_no_map).astype(np.float32)
    
    df['edjefe'] = df['edjefe'].replace(yes_no_map).astype(np.float32)
    df['edjefa'] = df['edjefa'].replace(yes_no_map).astype(np.float32)
    
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])

In [25]:
dprint("Encoding Data....")
encode_data(train)
encode_data(test)
dprint("Done...")

[2019-06-05 09:51] Encoding Data....
[2019-06-05 09:51] Done...


In [0]:
def do_features(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                 #('', '', ''),
                ]
    
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('people_weird_stat', 'tamhog', 'r4t3')]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)       
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
    
    # aggregation rules over household
    aggs_num = {'age': ['min', 'max', 'mean'],
                'escolari': ['min', 'max', 'mean']
               }
    aggs_cat = {'dis': ['mean']}
    for s_ in ['estadocivil', 'parentesco', 'instlevel']:
        for f_ in [f_ for f_ in df.columns if f_.startswith(s_)]:
            aggs_cat[f_] = ['mean', 'count']
    # aggregation over household
    for name_, df_ in [('18', df.query('age >= 18'))]:
        df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)
        df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])
        df = df.join(df_agg, how='left', on='idhogar')
        del df_agg
    # do something advanced above...
    
    # Drop SQB variables, as they are just squres of other vars 
    df.drop([f_ for f_ in df.columns if f_.startswith('SQB') or f_ == 'agesq'], axis=1, inplace=True)
    # Drop id's
    df.drop(['Id', 'idhogar'], axis=1, inplace=True)
    # Drop repeated columns
    df.drop(['hhsize', 'female', 'area2'], axis=1, inplace=True)
    return df

In [27]:
dprint("Do_feature Engineering....")
train = do_features(train)
test = do_features(test)
dprint("Done....")

[2019-06-05 09:52] Do_feature Engineering....
[2019-06-05 09:52] Done....


In [28]:
dprint("Fill Na value....")
train = train.fillna(0)
test = test.fillna(0)
dprint("Done....")

[2019-06-05 10:13] Fill Na value....
[2019-06-05 10:13] Done....


In [29]:
train.shape,test.shape


((9557, 221), (23856, 220))

In [0]:
cols_to_drop = [
    id_name, 
    target_name,
]
X = train.drop(cols_to_drop, axis=1, errors='ignore')
y = train[target_name].values


In [31]:
X.shape,y.shape

((9557, 220), (9557,))